In [27]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [9]:
 # 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Min Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,ushuaia,AR,-54.8000,-68.3000,50.00,50.00,62,40,34.52,light rain
1,1,kalmunai,LK,7.4167,81.8167,78.01,78.01,84,95,2.73,heavy intensity rain
2,2,khatanga,RU,71.9667,102.5000,-38.11,-38.11,97,17,9.69,few clouds
3,3,tuktoyaktuk,CA,69.4541,-133.0374,-13.00,-13.00,77,75,8.05,broken clouds
4,4,castro,BR,-24.7911,-50.0119,67.98,67.98,92,73,4.34,broken clouds


In [10]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


In [11]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
city_data_df_two = city_data_df.loc[(city_data_df["Max Temp"] < max_temp) & (city_data_df["Min Temp"] > min_temp)]
city_data_df_two

,City_ID,City,Country,Lat,Lng,Max Temp,Min Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,kalmunai,LK,7.4167,81.8167,78.01,78.01,84,95,2.73,heavy intensity rain
5,5,avarua,CK,-21.2078,-159.7750,82.40,82.40,78,20,12.66,light rain
6,6,salinopolis,BR,-0.6136,-47.3561,78.37,78.37,82,86,5.91,overcast clouds
9,9,elat,IL,29.5581,34.9482,77.00,77.00,29,0,4.61,clear sky
10,10,butaritari,KI,3.0707,172.7902,82.47,82.47,78,15,18.48,few clouds
...,...,...,...,...,...,...,...,...,...,...,...
684,684,inirida,CO,3.8653,-67.9239,83.08,83.08,76,89,1.79,overcast clouds
687,687,enugu,NG,6.4402,7.4943,77.61,77.61,78,100,2.04,overcast clouds
689,689,bara,NG,10.3744,10.7288,87.30,87.30,44,84,17.34,broken clouds
691,691,jeremie,HT,18.6500,-74.1167,77.70,77.70,77,38,3.65,scattered clouds


In [16]:
# 4a. Determine if there are any empty rows.
city_data_df_two.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Min Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [18]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
city_data_df_two = city_data_df_two.dropna()
city_data_df_two

,City_ID,City,Country,Lat,Lng,Max Temp,Min Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,kalmunai,LK,7.4167,81.8167,78.01,78.01,84,95,2.73,heavy intensity rain
5,5,avarua,CK,-21.2078,-159.7750,82.40,82.40,78,20,12.66,light rain
6,6,salinopolis,BR,-0.6136,-47.3561,78.37,78.37,82,86,5.91,overcast clouds
9,9,elat,IL,29.5581,34.9482,77.00,77.00,29,0,4.61,clear sky
10,10,butaritari,KI,3.0707,172.7902,82.47,82.47,78,15,18.48,few clouds
...,...,...,...,...,...,...,...,...,...,...,...
684,684,inirida,CO,3.8653,-67.9239,83.08,83.08,76,89,1.79,overcast clouds
687,687,enugu,NG,6.4402,7.4943,77.61,77.61,78,100,2.04,overcast clouds
689,689,bara,NG,10.3744,10.7288,87.30,87.30,44,84,17.34,broken clouds
691,691,jeremie,HT,18.6500,-74.1167,77.70,77.70,77,38,3.65,scattered clouds


In [20]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = city_data_df_two[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,kalmunai,LK,78.01,heavy intensity rain,7.4167,81.8167,
5,avarua,CK,82.40,light rain,-21.2078,-159.7750,
6,salinopolis,BR,78.37,overcast clouds,-0.6136,-47.3561,
9,elat,IL,77.00,clear sky,29.5581,34.9482,
10,butaritari,KI,82.47,few clouds,3.0707,172.7902,
13,hithadhoo,MV,83.19,overcast clouds,-0.6000,73.0833,
16,mahebourg,MU,78.80,moderate rain,-20.4081,57.7000,
23,vaini,TO,84.20,scattered clouds,-21.2000,-175.2000,
31,huarmey,PE,76.57,clear sky,-10.0681,-78.1522,
34,rikitea,PF,80.02,clear sky,-23.1203,-134.9692,


In [33]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    location = f"{lat},{lng}"
    
    params["location"]=f"{lat},{lng}"
        
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    try:
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except (IndexError):
        print("Hotel not found...skipping.")
        

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [34]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,kalmunai,LK,78.01,heavy intensity rain,7.4167,81.8167,VS Villa
5,avarua,CK,82.40,light rain,-21.2078,-159.7750,Paradise Inn
6,salinopolis,BR,78.37,overcast clouds,-0.6136,-47.3561,Salinas Park Resort Maçarico AP101
9,elat,IL,77.00,clear sky,29.5581,34.9482,Club Hotel Eilat
10,butaritari,KI,82.47,few clouds,3.0707,172.7902,Isles Sunset Lodge
...,...,...,...,...,...,...,...
684,inirida,CO,83.08,overcast clouds,3.8653,-67.9239,Fuente del Guania Hotel de Lujo
687,enugu,NG,77.61,overcast clouds,6.4402,7.4943,Bontera Suite
689,bara,NG,87.30,broken clouds,10.3744,10.7288,
691,jeremie,HT,77.70,scattered clouds,18.6500,-74.1167,Auberge’Inn


In [36]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].isnull().sum()

0

In [37]:
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,kalmunai,LK,78.01,heavy intensity rain,7.4167,81.8167,VS Villa
5,avarua,CK,82.40,light rain,-21.2078,-159.7750,Paradise Inn
6,salinopolis,BR,78.37,overcast clouds,-0.6136,-47.3561,Salinas Park Resort Maçarico AP101
9,elat,IL,77.00,clear sky,29.5581,34.9482,Club Hotel Eilat
10,butaritari,KI,82.47,few clouds,3.0707,172.7902,Isles Sunset Lodge
...,...,...,...,...,...,...,...
684,inirida,CO,83.08,overcast clouds,3.8653,-67.9239,Fuente del Guania Hotel de Lujo
687,enugu,NG,77.61,overcast clouds,6.4402,7.4943,Bontera Suite
689,bara,NG,87.30,broken clouds,10.3744,10.7288,
691,jeremie,HT,77.70,scattered clouds,18.6500,-74.1167,Auberge’Inn


In [39]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [40]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<d1>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [42]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))